# Smart signatures – Exercise
#### 06.1 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Write and deploy smart Signatures

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [2]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [3]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk.future import transaction
from algosdk.future.transaction import PaymentTxn
from algosdk.future.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.future.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import hashlib

In [4]:
from pyteal import *

In [5]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [6]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
5GIOBOLZSQEHTNNXWRJ6RGNPGCKWYJYUZZKY6YXHJVKFZXRB2YLDFDVH64


#### Check Purestake API

In [7]:
last_block = algod_client.status()["last-round"]
print(f"Last committed block is: {last_block}")

Last committed block is: 19823039


## Exercise
* Below is a copy of `06.1f_WSC_SmartSigngaures`
* Use this **as a starting point**
* Implement the following conditions
    * Bob can withdraw exactly 0.123 ALGOs
    * If he says "Thank You", he can withdraw up to 0.6 ALGOs
    * Alice can only withdraw amounts between (including) 0.5 and 0.9 ALGOs
    * Dina can withdraw whatever amount she wants, but only if she sends uses the code "WSC rocks"

**HINT** You will have to change quite a few things below

##### Step 1: The programmer writes down the conditions as a PyTeal program

In [9]:
max_amount = Int(int(1*1E6))                         # <---- 1e6 micro Algos = 1 Algo

withdraw_cond_1 =  And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() == Int(int(0.123*1e6)),             # Withdraws 0.123 Algo
)

withdraw_cond_2 =  And (
    Txn.receiver() == Addr(Bob["public"]),           # Receipient must be Bob
    Txn.amount() <= Int(int(0.6*1e6)),               # Withdraws max 0.6 Algo
    Txn.note() == Bytes('{"Thank You"}'),
)

withdraw_cond_3 =  And (
    Txn.receiver() == Addr(Alice["public"]),          # Receipient must be Alice
    Txn.amount() >= Int(int(0.5*1e6)),                # Withdraws between 0.5 Algo ...
    Txn.amount() <= Int(int(0.9*1e6)),                # ... and 0.9 Algo
)

withdraw_cond_4 =  And (
    Txn.receiver() == Addr(Dina["public"]),           # Receipient must be Dina
    Txn.note() == Bytes('{"WSC rocks"}'),
)

fee_cond =  (Txn.fee() <= Int(1000))                  # Fee attack

saftey_cond = And (
        Txn.type_enum() == TxnType.Payment,                     # Must be a "payment" transaction
        Global.group_size() == Int(1),                          # Only 1 Txn
        Txn.rekey_to() == Global.zero_address(),                # Rekey attack
        Txn.close_remainder_to () == Global.zero_address(),     # Closeout attack
)

homework_pyteal = And(
    fee_cond, 
    saftey_cond, 
    Or(withdraw_cond_1,withdraw_cond_2,withdraw_cond_3,withdraw_cond_4)
    )

##### Step 2: Compile PyTeal -> Teal

In [11]:
homework_teal = compileTeal(homework_pyteal, Mode.Signature, version=3)
print(homework_teal)

#pragma version 3
txn Fee
int 1000
<=
txn TypeEnum
int pay
==
global GroupSize
int 1
==
&&
txn RekeyTo
global ZeroAddress
==
&&
txn CloseRemainderTo
global ZeroAddress
==
&&
&&
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 123000
==
&&
txn Receiver
addr O2SLRPK4I4SWUOCYGGKHHUCFJJF5ORHFL76YO43FYTB7HUO7AHDDNNR5YA
==
txn Amount
int 600000
<=
&&
txn Note
byte "{\"Thank You\"}"
==
&&
||
txn Receiver
addr HITPAAJ4HKANMP6EUYASXDUTCL653T7QMNHJL5NODL6XEGBM4KBLDJ2D2E
==
txn Amount
int 500000
>=
&&
txn Amount
int 900000
<=
&&
||
txn Receiver
addr QAPHXKEK4Y3HR2Y46LYXVUDDVBA5HOBLH4UAKQ7G6IA6QJFEQFO7FEF73M
==
txn Note
byte "{\"WSC rocks\"}"
==
&&
||
&&


##### Step 3: Compile Teal -> Bytecode for AVM

In [ ]:
Modesty2 = algod_client.compile(modesty2_teal)
Modesty2

##### Step 4: Alice funds and deploys the smart signature

In [ ]:
# Step 1: prepare transaction
sp = algod_client.suggested_params()
amt = int(5.2*1e6)
txn = transaction.PaymentTxn(sender=Alice['public'], sp=sp, receiver=Modesty2['hash'], amt=amt)

# Step 2+3: sign and sen
stxn = txn.sign(Alice['private'])
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

##### Step 5: Alice informs Bob

In [ ]:
print("Alice communicates to Bob the following")
print("Compiled smart signature:", Modesty2['result'])
print("Address of smart signature: ", Modesty2['hash'])

#### Step 6: Charlie makes a small request

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(1*1e6)              # <---------- OK!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Charlie['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Dina makes a large request

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- too much for Dina!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Dina['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
try:
    txid = algod_client.send_transaction(stxn)
except algosdk.error.AlgodHTTPError as err:
    print(err)               # print entire error message

# Step 4: wait for confirmation
# No need to wait for confirmation
#txinfo = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob makes a large request
* This is OK, because it is Bob!

In [ ]:
# Step 1: prepare TX
sp = algod_client.suggested_params()
withdrawal_amt = int(2*1e6)              # <---------- ok for Bob!!
txn = PaymentTxn(sender=Modesty2['hash'], sp=sp, 
                 receiver=Bob['public'], amt=withdrawal_amt)

# Step 2: sign TX <---- This step is different!
encodedProg = Modesty2['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)


# Step 3: send
txid = algod_client.send_transaction(stxn)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)